In [1]:
import os, pickle

import numpy as np
import pandas as pd

from tqdm import tqdm

from collections import defaultdict, Counter

import cudf, itertools

In [2]:
train = cudf.read_parquet('../data/train.parquet')
test = cudf.read_parquet('../data/test.parquet')

data = cudf.concat([train, test])

In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
type_weight = {0:1, 1:6, 2:3}

In [7]:
version = 1
data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

# print len of sessions
print(len(sessions))

chunk_size = 100_000

tmp = None
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.sort_values(['session','ts'],ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n', axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    # df shape on current iteration
    print(df.shape)

    if tmp is None: tmp = df
    else: tmp.add(df, fill_value=0)
    
    # tmp shape on current iteration
    print(tmp.shape)

    if i>10:
        break

# # CONVERT MATRIX TO DICTIONARY
# tmp = tmp.reset_index()
# tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
# # SAVE TOP 40
# tmp = tmp.reset_index(drop=True)
# tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
# tmp = tmp.loc[tmp.n<15].drop('n',axis=1)
# print(tmp.shape)
# # SAVE TO DISK
# df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
# with open(f'../data/top_15_carts_orders_v{version}.pkl', 'wb') as f:
#     pickle.dump(df.to_dict(), f)

: 

: 

In [4]:
version = 1
data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

chunk_size = 100_000

tmp = None
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.loc[df.type.isin([1, 2])]

    df = df.sort_values(['session','ts'], ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df, on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = 1
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    if tmp is None: tmp = df
    else: tmp.add(df, fill_value=0)


# CONVERT MATRIX TO DICTIONARY
tmp = tmp.reset_index()
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
# SAVE TOP 15
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<15].drop('n',axis=1)
# SAVE TO DISK
df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
with open(f'../data/top_15_buy2buy_v{version}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)

100%|██████████████████████████████████████████████| 146/146 [00:57<00:00,  2.53it/s]


In [5]:
version = 1
data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

chunk_size = 100_000

tmp = None
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.sort_values(['session','ts'], ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    if tmp is None: tmp = df
    else: tmp.add(df, fill_value=0)


# CONVERT MATRIX TO DICTIONARY
tmp = tmp.reset_index()
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
# SAVE TOP 40
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<20].drop('n',axis=1)
# SAVE TO DISK
df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
with open(f'../data/top_20_clicks_v{version}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)

100%|██████████████████████████████████████████████| 146/146 [02:39<00:00,  1.10s/it]


In [6]:
display(test)

,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0
...,...,...,...,...
6928118,14571577,1141710,1662328774,0
6928119,14571578,519105,1662328775,0
6928120,14571579,739876,1662328775,0
6928121,14571580,202353,1662328781,0


In [7]:
display(train)

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
216716091,12899776,1737908,1661723987,0
216716092,12899777,384045,1661723976,0
216716093,12899777,384045,1661723986,0
216716094,12899778,561560,1661723983,0


In [20]:
%%time
# LOAD THREE CO-VISITATION MATRICES

VER = 1

top_20_clicks = pd.read_pickle(f'../data/top_20_clicks_v{VER}.pkl')
top_15_carts_orders = pd.read_pickle(f'../data/top_15_carts_orders_v{VER}.pkl')
top_15_buy2buy = pd.read_pickle(f'../data/top_15_buy2buy_v{VER}.pkl')

# TOP CLICKS AND ORDERS IN TEST
top_clicks = test.loc[test['type']==0,'aid'].value_counts().index.values[:20]
top_orders = test.loc[test['type']==2,'aid'].value_counts().index.values[:20]

# print shape of each matrix
print(f'clicks: {len(top_20_clicks)}')
print(f'carts: {len(top_15_carts_orders)}')
print(f'buy2buy: {len(top_15_buy2buy)}')

clicks: 374915
carts: 362351
buy2buy: 138144
CPU times: user 1.11 s, sys: 88.6 ms, total: 1.2 s
Wall time: 1.33 s


In [24]:
top_20_clicks[1]

[553507,
 4541,
 1442126,
 190430,
 1174094,
 757471,
 1496186,
 504137,
 28092,
 1180838,
 980751,
 1271754,
 1837463,
 930325,
 526193,
 977455,
 23704,
 1142186,
 458410,
 675264]